2. Using a dictionary in qdapDictionaries, conduct sentiment analysis on your corpus. Write up the results in a pdf and interpret them.

Submit the answers to both assignment questions in a single pdf. file.

# Setup

In [1]:
import os as os
import sys as sys
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import rc
from pylab import rcParams
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

#torch goodness
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

#huggingface goodness
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

#set graph config
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

#set seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

#check gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# load training datasets

In [3]:
# imdb from http://ai.stanford.edu/~amaas/data/sentiment/
#"We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing."
#this file was larger than 100mb (github's file size limit)
df_imdb = pd.read_csv('imdb.csv')

#https://www.kaggle.com/lava18/google-play-store-apps
#google play app reviews ~16k
df_google = pd.read_csv("reviews.csv")

Crystal Feel http://www.crystalfeel.socialanalyticsplus.net/ 
Crystal Feel Manual http://52.3.21.155/crystalfeel/[CrystalFeel]_User_Manual.pdf
Super Cool App Thingy https://medium.com/@mirzamujtaba10/sentiment-analysis-642b935ab6f9